In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode ="nearest",
    horizontal_flip = True,
    validation_split=0.15)
train_generator = train_datagen.flow_from_directory(
    "/kaggle/input/datavision/train2",
    target_size=(224,224),
    batch_size=16,
    class_mode='categorical',
    subset='training')
validation_generator = train_datagen.flow_from_directory(
    "/kaggle/input/datavision/train2", 
    target_size=(224,224),
    batch_size=16,
    class_mode='categorical',
    subset='validation')

In [ ]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer,GlobalAveragePooling1D
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from keras.applications.mobilenet import MobileNet
model1 = MobileNet(include_top=False , weights='imagenet',input_shape=(224,224,3))
for layer in model1.layers[0:15]:
    layer.trainable=False
for layer in model1.layers[15:]:
    layer.trainable=True
#model1.trainable=False  
x=Flatten()(model1.output)
output=Dense(5,activation='softmax')(x)
model=Model(inputs=model1.input,outputs=output)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit_generator(train_generator, epochs=100,validation_data=validation_generator)

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
img=cv2.imread('/kaggle/input/pictures/angry.jpeg',cv2.IMREAD_COLOR)
plt.imshow(img)
img=img/255.0
image=cv2.resize(img,(224,224))
image=np.array(image).reshape(-1,224,224,3)
print(model.predict(image))

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
img=cv2.imread('/kaggle/input/pictures/sad.jpg',cv2.IMREAD_COLOR)
plt.imshow(img)
img=img/255.0
image=cv2.resize(img,(224,224))
image=np.array(image).reshape(-1,224,224,3)
print(model.predict(image))

In [ ]:
from keras.models import model_from_json
model_json = model.to_json()
with open("modelexpression.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("modelexpression.h5")
print("Saved model to disk")    